In [1]:
import numpy as np
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml

In [2]:
class TiedCovClassifier:
    def __init__(self):
        self.mu = {}

    def train(self, DTR, LTR):
        self.mu, C = ml.MU_Cov_calculator(DTR, LTR)
        N = DTR.shape[1]
        self.C_ = numpy.zeros((DTR.shape[0], DTR.shape[0]))
        for i in numpy.unique(LTR):
            Nc = DTR[:, LTR == i].shape[1]
            self.C_ += Nc*C[i]
        self.C_ /= N


    def test(self, DTE, LTE):
        S = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        predicted = []

        for i in numpy.unique(LTE):
            S[i, :] = ml.GAU_logpdf_ND(DTE, self.mu[i], self.C_) + numpy.log(1 / 2)

        Sp = scipy.special.logsumexp(S, axis=0)

        for x, p in zip(S.T, Sp):
            tmp = x - p
            predicted.append(numpy.argmax(tmp))

        predicted = numpy.array(predicted)

        True_prediction = numpy.array([predicted == LTE])

        error = 1 - (numpy.count_nonzero(True_prediction) / True_prediction.size)

        print("TiedCovClassifier error:", error)


In [3]:
Data, label = ml.loadFile('../Train.txt')
(DTR, LTR), (DTE,LTE) = ml.split_db_2to1(Data, label, seed=42)

In [4]:
G=TiedCovClassifier()
G.train(DTR,LTR)
G.test(DTE,LTE)

TiedCovClassifier error: 0.023849512932482386


In [5]:
Gn=TiedCovClassifier()
Gn.train(ml.z_normalization(DTR), LTR)
Gn.test(DTE, LTE)
Gn.test(ml.z_normalization(DTE), LTE)

TiedCovClassifier error: 0.9287873698354048
TiedCovClassifier error: 0.023513604299630453


In [6]:
GG=TiedCovClassifier()
GG.train(ml.gaussianize(DTR), LTR)
GG.test(DTE, LTE)
GG.test(ml.gaussianize(DTE), LTE)

TiedCovClassifier error: 0.9076251259657373
TiedCovClassifier error: 0.060127645280483755


In [7]:
class TiedCovClassifier_mod1:
    def __init__(self):
        self.mu = {}

    def train(self, DTR, LTR):
        self.mu, C = ml.MU_Cov_calculator(DTR, LTR)
        N = DTR.shape[1]
        self.C_ = numpy.zeros((DTR.shape[0], DTR.shape[0]))
        for i in numpy.unique(LTR):
            Nc = DTR[:, LTR == i].shape[1]
            self.C_ += Nc*C[i]
        self.C_ /= N


    def test(self, DTE, LTE, app):
        S = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        ll = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        predicted = []

        for i in numpy.unique(LTE):
            ll[i, :] = ml.GAU_logpdf_ND(DTE, self.mu[i], self.C_)
            S[i, :] = ll[i, :]  + numpy.log(1/2)
       
        Sp = scipy.special.logsumexp(S, axis=0)

        for x, p in zip(S.T, Sp):
            tmp = x - p
            predicted.append(numpy.argmax(tmp))

        predicted = numpy.array(predicted)

        app_bayes_risk=ml.compute_Bayes_risk(ml.compute_confusion_matrix(predicted, LTE), app)
        DCF = ml.compute_norm_Bayes(app_bayes_risk, app)
        
        llr = numpy.array(ll[1, :]-ll[0, :])
        minDCF= ml.compute_min_DCF(llr, app, LTE)

        True_prediction = numpy.array([predicted == LTE])
        error = 1 - (numpy.count_nonzero(True_prediction) / True_prediction.size)
      
        print("\-/ \-/ \-/ \-/ \-/ ")
        print("Tied Covariance Gaussian Classifier error:", error)
        print(app,"DCF:", DCF, "minDCF:", minDCF)
        print("/-\ /-\ /-\ /-\ /-\ ")

In [8]:
G=TiedCovClassifier_mod1()
G.train(DTR,LTR)
G.test(DTE,LTE, [1/2,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.023849512932482386
[0.5, 1, 1] DCF: 0.1993890047776058 minDCF: 0.10910840454881907
/-\ /-\ /-\ /-\ /-\ 


In [10]:
G.test(DTE,LTE, [0.1,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.023849512932482386
[0.1, 1, 1] DCF: 0.2526828611802705 minDCF: 0.2426606554067694
/-\ /-\ /-\ /-\ /-\ 


In [11]:
G.test(DTE,LTE, [0.7,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.023849512932482386
[0.7, 1, 1] DCF: 0.45635870174730264 minDCF: 0.20784424556445277
/-\ /-\ /-\ /-\ /-\ 


In [13]:
G.test(DTE,LTE, [0.6,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.023849512932482386
[0.6, 1, 1] DCF: 0.2957526411412421 minDCF: 0.15026848798869524
/-\ /-\ /-\ /-\ /-\ 
